In [7]:
import numpy as np
import pandas as pd
import string
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

In [19]:
d1 = pd.read_csv(r'/home/chinchu/Desktop/Cyber bullying_tweets/cyberbullying_tweets.csv')

stop_words = set(stopwords.words('english'))
def clear(text):
    text = re.sub(r'@[A-Za-z0-9]+','',text)
    text = re.sub(r'RT[\s]+','',text)
    text = re.sub(r'https?:\/\/\S+','',text)
    text = re.sub(r'#','',text)

    return text

d1['tweet_text'] = d1['tweet_text'].apply(clear)
d1['tweet_text'] = d1['tweet_text'].str.replace("[^a-zA-Z@]",' ')

d1['tweet_text'] = d1['tweet_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))

tokenizer = RegexpTokenizer(r'\w+')
d1['tweet_text'] = d1['tweet_text'].apply(lambda x: tokenizer.tokenize(x.lower()))


stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = " ".join([stemmer.stem(i) for i in text])
    return stem_text

d1['tweet_text'] = d1['tweet_text'].apply(lambda x: word_stemmer(x))


lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = [lemmatizer.lemmatize(i) for i in text]
    return lem_text

d1['tweet_text'].apply(lambda x: word_lemmatizer(x))


0        [o, t, h, e, r,  , w, o, r, d,  , k, a, t, a, ...
1        [a, u, s, s, i, e, t, v,  , w, h, i, t, e,  , ...
2        [c, l, a, s, s, i,  , w, h, o, r, e,  , m, o, ...
3        [_, g, i, o,  , m, e, h,  , t, h, a, n, k,  , ...
4        [t, h, i,  , i, s, i,  , a, c, c, o, u, n, t, ...
                               ...                        
47687    [b, l, a, c, k,  , a, r, e, n,  , t,  , e, x, ...
47688    [t, u, r, n, e, r,  , w, i, t, h, h, o, l, d, ...
47689    [s, w, e, a, r,  , g, o, d,  , t, h, i,  , d, ...
47690    [f, u, c, k,  , y, o, u, r,  , n, i, g, g, e, ...
47691    [b, r, o,  , g, o, t, t, a,  , c, h, i, l, l, ...
Name: tweet_text, Length: 47692, dtype: object

In [20]:
d1.head()

,tweet_text,cyberbullying_type
0,other word katandandr your food crapilici,not_cyberbullying
1,aussietv white theblock imacelebrityau today s...,not_cyberbullying
2,classi whore more velvet cupcak,not_cyberbullying
3,_gio meh thank head concern about anoth angri ...,not_cyberbullying
4,thi isi account pretend kurdish account like i...,not_cyberbullying


In [21]:
d1['tweet_text'].duplicated().sum()

3035

In [22]:
d1.drop_duplicates('tweet_text', inplace = True)

In [23]:
d1['cyberbullying_type'] = d1['cyberbullying_type'].replace(to_replace=['gender','other_cyberbullying','age','ethnicity'],value = 'Bullying')
d1['cyberbullying_type'] = d1['cyberbullying_type'].map({'not_cyberbullying':0,'Bullying':1})

In [24]:
d1['cyberbullying_type'].fillna(0,inplace=True)
d1['tweet_text'].fillna('',inplace=True)

/tmp/ipykernel_8817/2852598885.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  d1['cyberbullying_type'].fillna(0,inplace=True)
/tmp/ipykernel_8817/2852598885.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

In [25]:
X = d1['tweet_text']  
y = d1['cyberbullying_type'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)


tfidf = TfidfVectorizer(max_features = 5000)
X_train_tf = tfidf.fit_transform(X_train)
X_test_tf = tfidf.transform(X_test)  

print(X_train_tf.shape)
print(X_test_tf.shape)

(40191, 5000)
(4466, 5000)


In [26]:
import pickle

In [ ]:
with open('tfidf_vectorization','wb') as f:
    pickle.dump(tfidf,f)

In [27]:
logic = LogisticRegression()
logic.fit(X_train_tf,y_train)
pred = logic.predict(X_test_tf)

In [ ]:
with open('model','wb') as f:
    pickle.dump(logic,f)